# 1. Pk를 사용해서 add, update를 함께 사용하기
#### get 방식 : form 띄우기
- add할 때는 빈 칸 띄우기
- 수정할 때는 이전 데이터 띄우기

In [ ]:
#views.py
'''
# 특정 post를 수정해야 함 -> 파라미터를 받아야 함(pk= id)
class PostEditView(View):
    def get(self, request, pk):
        # add할 때
        if pk == 0:
            form = PostForm()

        # pk값이 있을 때(수정)
        else:
            post = get_object_or_404(Post, pk = pk)
            form = PostForm(initial = {'title': post.title, 'text': post.text})
        return render(request, 'blog/edit.html', {'form': form ,'pk':pk})

    def post(self, request, pk):
        
        form = PostForm(request.POST)
        # validator를 통화하면 처리
        if form.is_valid():
            post = get_object_or_404(Post, pk = pk)
            post.title = form['title'].value()
            post.text = form['text'].value()
            post.publish()

            return redirect('list')
        return render(request, 'blog/edit.html', {'form': form ,'pk':pk})
'''

## 트릭: action값을 빼면 -> 자기 자신이 호출됨

> 수정전:
- action값을 add, edit를 줌
- add일 때는 pk가 반드시 필요하기 때문에 action에 pk값을 전달해줌.
- 주소값을 pk를 사용할 수 있도록 만듦.

In [ ]:
# edit.html 수정 전
'''
<form action="{% url 'edit' pk %}" method=post>
    {% csrf_token %}
 
        {{ form.as_p }}
 
    <input type="submit" value="작성" >
 
 </form>
 


'''

> 수정후
- action을 제거하면 자기 자신을 호출
- 이미 url에 pk값을 갖고 있기 때문에 action에서 pk값을 따로 전달할 필요가 없음.
- 굳이 pk를 명시하지 않아도 pk를 전달할 수 있음.

In [ ]:
# edit.html수정 후
'''
<script src="http://code.jquery.com/jquery-1.11.3.min.js"></script>
<script src="http://code.jquery.com/jquery-migrate-1.2.1.min.js"></script>

 <style>
      .bg { background-color: #eeeeee; }
      .bd { border: 1px solid #666666; }
 </style>

{% if form.title.value %}
<h1>  수정하기 </h1>
{% else %}
<h1>  신규작성</h1>
{% endif %}


<form method=post>
   {% csrf_token %}
   	{{ form.as_p }}

   <input type="submit" value="작성" >

</form>

<script>
   $("#id_title").addClass('bg bd');
</script>


'''

# 2. form 객체를 꾸미기

> html을 사용
- 수동적이지만, 자유도가 높음.
> form 사용
- 편리하지만, 우리가 생성한 것이 아니라 자유도가 낮음 -> jquery 사용

In [ ]:
#  edit.html
'''
<script src="http://code.jquery.com/jquery-1.11.3.min.js"></script>
<script src="http://code.jquery.com/jquery-migrate-1.2.1.min.js"></script>

 <style>
      .bg { background-color: #eeeeee; }
      .bd { border: 1px solid #666666; }
 </style>

{% if form.title.value %}
<h1>  수정하기 </h1>
{% else %}
<h1>  신규작성</h1>
{% endif %}


<form method=post>
   {% csrf_token %}
   	{{ form.as_p }}

   <input type="submit" value="작성" >

</form>

<script>
    // table에서 id의 title 노드에 bg, bd 속성 적용하기
   $("#id_title").addClass('bg bd');
</script>

'''

# 3. update할 때 일일이 값을 넣기 어려움
- db와 form이 영향이 없기 때문에(독립적)
- 하나하나 매핑해야 함.

In [ ]:
# 수정전 Form class 상속 받았을 때
'''
class PostForm(Form):
    title = CharField(label = '제목', max_length = 20, validators=[validator])
    text = CharField(label = '내용', widget = Textarea) # widget: ui를 의미
'''

In [3]:
# 수정전 vies.py


'''
post = get_object_or_404(Post, pk = pk)
form = PostForm(initial = {'title': post.title, 'text': post.text})

'''

'''
Post.objects.create(title = form['title'].value(), text = form['text'].value(), author = user)


'''

"\nPost.objects.create(title = form['title'].value(), text = form['text'].value(), author = user)\n\n\n"

### form과 model 합리기
- form으로 상속받지 않고, ModelForm에서 상속받기

### ModelForm class
- 일일이 form tag를 만들 필요 없다
- 내가 필요한 fiele값만 갖고와서 사용할 수 있다
- 주의: 장고model을 사용해서 dab를 구성하는 경우에만 사용가능
- 이미 db가 생성되어있어서 장고 orm을 사용할 수 없는 경우에는 그냥 form class를 상속받아서 사용해야 함.

In [ ]:
# 수정후: 
'''
def PostForm(formsModelForm):
        # 부가적인 정보를 입력하는 calss
        class Meta:
            # model: 정해진 변수명
            model = Post
            filed = ['title', 'text']

'''

### ModelForm에서 validator 설정하기

In [ ]:
'''
# ModelForm class를 상속받았을 때
def PostForm(formsModelForm):
        # 부가적인 정보를 입력하는 calss
        class Meta:
            # model: 정해진 변수명
            model = Post
            filed = ['title', 'text']
        def __init__(self, *args, **kwargs):
            super(PostForm, self).__init__(*args, **kwargs)
            self.fileds['title'].validator=[validator]
'''

# 4. forms.py 따로 관리하기
- forms에 관련된 코드를 분리해서 관리하기
- 관행적으로 forms, views, models.py가 있음

In [ ]:
#forms.py
'''

from . import models
from django.forms import ValidationError
from django import forms


def validator(value):
    if len(value) < 5 : raise  ValidationError('길이가 너무 짧아요')


class PostForm(forms.ModelForm):
        # 부가적인 정보를 입력하는 calss
    class Meta:
        # model: 정해진 변수명
        model = models.Post
        fields = ['title', 'text']
        
    def __init__(self, *args, **kwargs):
        super(PostForm, self).__init__(*args, **kwargs)
        self.fields['title'].validator=[validator]


'''

In [ ]:
# views.py
'''
# ModelForm class를 상속받았을 때
class PostEditView(View) :
    def get(self, request, pk):
        if pk == 0 :
            form = forms.PostForm()
        else :
            post = get_object_or_404(models.Post, pk=pk)
            form = forms.PostForm(instance=post)
        return render(request, "blog/edit.html", {"form":form})

    def post(self, request, pk):

        username = request.session["username"]
        user = User.objects.get(username=username)

        if pk == 0:
            form = forms.PostForm(request.POST)
        else:
            post = get_object_or_404(Post, pk=pk)
            form = forms.PostForm(request.POST, instance=post)

        if form.is_valid():
            post = form.save(commit=False)
            if pk == 0:
                post.author = user
                post.save()
            else :
                post.publish()
            return redirect("list")
        return render(request, "blog/edit.html", {"form": form})
            
          

'''

# 5. 한번 더 모듈화하기

> 함수로 구현한 것
- get 방식으로 호출하도록 구현했음.

###  url를 통일하기
- <int: pk> / <mode>로 만들기
    

> get 방식으로 호출할때 규칙
-  리스트를 출력할 때: 0/list
-  5번 포스트 보여줘: 5/detail
- 신규 데이터 작성: 0/add   -? 버튼 누르면 post 발생
- 5번 포스트 수정: 5/edit   -> 버튼 누르면 post 발생

## 5.1. get 방식부터 (클라이언트에서 요청할 때)

In [ ]:
# urls.py
'''
urlpatterns = [
    path('', views.index),
    path('login/', views.LoginView.as_view(), name ='login'),

    # 최종버전
    # 이 뷰가 실행되기 위해서는 파라미터가 2개 있어야 함!
    path('<int:pk>/<mode>', views.PostEditView.as_view(), name = 'edit')
]

'''

In [ ]:
# views.py
'''
class PostEditView(View) :
    def get(self, request, pk, mode):
        print(pk, mode)
        if mode =='add':
            # pk값은 상관없음.
            form = forms.PostForm()
        elif mode == 'list':
            # pk값 상관없음.
            username = request.session.get('username', '')
            user = User.objects.get(username =username)
            data= models.Post.objects.all().filter(author = user)
            context = {'data': data,'username': username}
            return render (request,'blog/list.html', context)
        elif mode =='detail':
            p = get_object_or_404(Post, pk = pk)
            return render(request, 'blog/detail.html', {'d': p})
        elif mode == 'edit':
            post = get_object_or_404(models.Post, pk=pk)
            form = forms.PostForm(instance=post)
        elif mode =='delete':
            post = get_object_or_404(models.Post, pk=pk)
            post.delete()
            return redirect('/0/list')
        else:
            return HttpResponse('mode를 잘 입력하세용')
        return render(request, "blog/edit.html", {"form":form})



'''

In [ ]:
# list.html
'''
<!-- import, includ 개념 -->
{% extends 'blog/base.html' %}


<!-- base template안의 위치 지정 -->

{% block content %}
    <br>
    <!-- urls.py에 잇는 패턴: edit 밖에 없음.
     생성되는 url: blog/0/add/ -->
    <a href="{% url 'edit' 0 'add' %}">Add </a> <br>
    여기부터 sub-template<br>

    {% for d in data%}
        <!-- 생성되는 url: /bolg/5/detail -->
    
            <a href="{url 'edit' d.pk 'detail' %}">{{d.title}}</a>
       <br>
    {%endfor%}


    

{% endblock %}



'''